# Scryfall Mechanics Catalog Exploration

**Objective:** Explore and validate Scryfall's mechanics catalog endpoints (keyword abilities, keyword actions, and ability words) for MTG card scoring system integration.

**Status:** Phase 0 - API Testing & Validation ONLY

**Purpose:** Confirm we can reliably fetch mechanics from Scryfall before designing MongoDB schema or integrating into the scoring system.

## Setup & Dependencies

Import required libraries and establish connectivity to the Scryfall API. Create a helper function to fetch catalog data.

In [1]:
import requests
import json
import pandas as pd
from collections import Counter
import re

# Scryfall API base URL
SCRYFALL_API = "https://api.scryfall.com"

# Test basic connectivity
response = requests.get(f"{SCRYFALL_API}/catalog/card-names", params={'limit': 1})
if response.status_code == 200:
    print("✅ Scryfall API is reachable")
else:
    print(f"❌ Scryfall API error: {response.status_code}")

✅ Scryfall API is reachable


In [2]:
def fetch_catalog(catalog_name):
    """Fetch a Scryfall catalog by name"""
    url = f"{SCRYFALL_API}/catalog/{catalog_name}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        return {
            'success': True,
            'catalog_name': catalog_name,
            'data': data.get('data', []),
            'total_values': data.get('total_values', 0),
            'object': data.get('object'),
            'uri': data.get('uri')
        }
    except Exception as e:
        return {
            'success': False,
            'catalog_name': catalog_name,
            'error': str(e)
        }

print("✅ fetch_catalog() helper function created")

✅ fetch_catalog() helper function created


## Fetch Keyword Abilities Catalog

Call the keyword-abilities endpoint to retrieve all keyword abilities. Display total count, sample mechanics, and verify data consistency.

In [3]:
# Fetch keyword abilities
abilities_result = fetch_catalog('keyword-abilities')

if abilities_result['success']:
    abilities = abilities_result['data']
    print(f"✅ Keyword Abilities: {len(abilities)} total")
    print(f"\n📊 Sample (first 20):")
    for i, ability in enumerate(abilities[:20], 1):
        print(f"  {i}. {ability}")
    
    print(f"\n📊 Sample (last 10):")
    for ability in abilities[-10:]:
        print(f"  - {ability}")
else:
    print(f"❌ Error fetching keyword abilities: {abilities_result['error']}")

✅ Keyword Abilities: 216 total

📊 Sample (first 20):
  1. Living weapon
  2. Jump-start
  3. Commander ninjutsu
  4. Legendary landwalk
  5. Nonbasic landwalk
  6. Megamorph
  7. Haunt
  8. Forecast
  9. Graft
  10. Fortify
  11. Frenzy
  12. Gravestorm
  13. Hideaway
  14. Level Up
  15. Infect
  16. Reach
  17. Rampage
  18. Phasing
  19. Multikicker
  20. Morph

📊 Sample (last 10):
  - Fear
  - Tiered
  - Mobilize
  - Double team
  - Job select
  - Mayhem
  - Web-slinging
  - Prowl
  - Solved
  - Firebending


## Fetch Keyword Actions Catalog

Call the keyword-actions endpoint to retrieve all keyword actions. Display total count and sample mechanics.

In [4]:
# Fetch keyword actions
actions_result = fetch_catalog('keyword-actions')

if actions_result['success']:
    actions = actions_result['data']
    print(f"✅ Keyword Actions: {len(actions)} total")
    print(f"\n📊 Sample (first 20):")
    for i, action in enumerate(actions[:20], 1):
        print(f"  {i}. {action}")
    
    print(f"\n📊 Sample (last 10):")
    for action in actions[-10:]:
        print(f"  - {action}")
    
    # Check if proliferate is included
    if 'Proliferate' in actions or 'proliferate' in actions:
        print("\n✅ CONFIRMED: 'Proliferate' found in keyword actions")
    else:
        print("\n⚠️ WARNING: 'Proliferate' not found - check casing")
        
else:
    print(f"❌ Error fetching keyword actions: {actions_result['error']}")

✅ Keyword Actions: 70 total

📊 Sample (first 20):
  1. Seek
  2. Activate
  3. Attach
  4. Cast
  5. Counter
  6. Create
  7. Destroy
  8. Discard
  9. Double
  10. Exchange
  11. Exile
  12. Adapt
  13. Support
  14. Play
  15. Regenerate
  16. Reveal
  17. Sacrifice
  18. Shuffle
  19. Tap
  20. Untap

📊 Sample (last 10):
  - Plot
  - Heist
  - Forage
  - Manifest dread
  - Endure
  - Waterbend
  - Airbend
  - Earthbend
  - Blight
  - Behold

✅ CONFIRMED: 'Proliferate' found in keyword actions


## Fetch Ability Words Catalog

Call the ability-words endpoint to retrieve all ability words. Display total count and verify presence of common ability words.

In [5]:
# Fetch ability words
ability_words_result = fetch_catalog('ability-words')

if ability_words_result['success']:
    ability_words = ability_words_result['data']
    print(f"✅ Ability Words: {len(ability_words)} total")
    print(f"\n📊 Sample (first 20):")
    for i, word in enumerate(ability_words[:20], 1):
        print(f"  {i}. {word}")
    
    print(f"\n📊 Sample (last 10):")
    for word in ability_words[-10:]:
        print(f"  - {word}")
    
    # Check for common ability words
    common_ability_words = ['Landfall', 'Raid', 'Metalcraft', 'Battalion']
    for word in common_ability_words:
        if word in ability_words:
            print(f"  ✅ {word} found")
        else:
            print(f"  ⚠️ {word} not found")
            
else:
    print(f"❌ Error fetching ability words: {ability_words_result['error']}")

✅ Ability Words: 66 total

📊 Sample (first 20):
  1. Eerie
  2. Battalion
  3. Bloodrush
  4. Channel
  5. Chroma
  6. Cohort
  7. Constellation
  8. Converge
  9. Delirium
  10. Domain
  11. Fateful hour
  12. Ferocious
  13. Formidable
  14. Grandeur
  15. Hellbent
  16. Heroic
  17. Imprint
  18. Inspired
  19. Join forces
  20. Kinship

📊 Sample (last 10):
  - Celebration
  - Paradox
  - Disappear
  - Will of the Planeswalkers
  - Survival
  - Flurry
  - Valiant
  - Start your engines!
  - Renew
  - Vivid
  ✅ Landfall found
  ✅ Raid found
  ✅ Metalcraft found
  ✅ Battalion found


## Data Quality Analysis

Combine all mechanics into a pandas DataFrame with computed fields and generate summary statistics.

In [6]:
# Combine all mechanics into one DataFrame for analysis
all_mechanics = []

if abilities_result['success']:
    for ability in abilities_result['data']:
        all_mechanics.append({
            'mechanic': ability,
            'category': 'keyword_ability',
            'length': len(ability),
            'has_space': ' ' in ability,
            'has_hyphen': '-' in ability,
            'word_count': len(ability.split())
        })

if actions_result['success']:
    for action in actions_result['data']:
        all_mechanics.append({
            'mechanic': action,
            'category': 'keyword_action',
            'length': len(action),
            'has_space': ' ' in action,
            'has_hyphen': '-' in action,
            'word_count': len(action.split())
        })

if ability_words_result['success']:
    for word in ability_words_result['data']:
        all_mechanics.append({
            'mechanic': word,
            'category': 'ability_word',
            'length': len(word),
            'has_space': ' ' in word,
            'has_hyphen': '-' in word,
            'word_count': len(word.split())
        })

df = pd.DataFrame(all_mechanics)

print("📊 DATA QUALITY ANALYSIS")
print(f"\nTotal mechanics: {len(df)}")
print(f"\nBreakdown by category:")
print(df['category'].value_counts())

print(f"\n📏 String length statistics:")
print(df.groupby('category')['length'].describe())

print(f"\n🔤 Multi-word mechanics:")
multi_word = df[df['word_count'] > 1]
print(f"  Total: {len(multi_word)}")
print(f"  Examples:")
for _, row in multi_word.head(10).iterrows():
    print(f"    - {row['mechanic']} ({row['category']})")

print(f"\n➖ Mechanics with hyphens:")
hyphenated = df[df['has_hyphen']]
print(f"  Total: {len(hyphenated)}")
print(f"  Examples:")
for _, row in hyphenated.head(10).iterrows():
    print(f"    - {row['mechanic']} ({row['category']})")

📊 DATA QUALITY ANALYSIS

Total mechanics: 352

Breakdown by category:
category
keyword_ability    216
keyword_action      70
ability_word        66
Name: count, dtype: int64

📏 String length statistics:
                 count      mean       std  min  25%  50%   75%   max
category                                                             
ability_word      66.0  8.893939  3.965733  4.0  6.0  8.0  10.0  25.0
keyword_ability  216.0  8.310185  3.158022  4.0  6.0  8.0  10.0  23.0
keyword_action    70.0  7.742857  4.402521  3.0  5.0  7.0   8.0  30.0

🔤 Multi-word mechanics:
  Total: 47
  Examples:
    - Living weapon (keyword_ability)
    - Commander ninjutsu (keyword_ability)
    - Legendary landwalk (keyword_ability)
    - Nonbasic landwalk (keyword_ability)
    - Level Up (keyword_ability)
    - Double strike (keyword_ability)
    - Cumulative upkeep (keyword_ability)
    - First strike (keyword_ability)
    - Aura Swap (keyword_ability)
    - Battle Cry (keyword_ability)

➖ Mechanics 

## Casing & Normalization Analysis

Analyze casing patterns and check for duplicate mechanics using case-insensitive comparison.

In [7]:
# Check casing patterns
print("📊 CASING ANALYSIS")

casing_examples = []
for mechanic in df['mechanic'].head(30):
    casing_examples.append({
        'original': mechanic,
        'lowercase': mechanic.lower(),
        'uppercase': mechanic.upper(),
        'title_case': mechanic.title(),
        'is_capitalized': mechanic[0].isupper() if mechanic else False
    })

casing_df = pd.DataFrame(casing_examples)
print("\n📝 First 15 mechanics - casing patterns:")
print(casing_df[['original', 'lowercase', 'is_capitalized']].head(15).to_string(index=False))

# Check for duplicate mechanics (case-insensitive)
lowercase_mechanics = [m.lower() for m in df['mechanic']]
duplicate_check = Counter(lowercase_mechanics)
duplicates = {k: v for k, v in duplicate_check.items() if v > 1}

if duplicates:
    print(f"\n⚠️ WARNING: Found {len(duplicates)} duplicate mechanics (case-insensitive):")
    for mech, count in duplicates.items():
        print(f"  - {mech}: appears {count} times")
else:
    print("\n✅ No duplicate mechanics found (case-insensitive)")

📊 CASING ANALYSIS

📝 First 15 mechanics - casing patterns:
          original          lowercase  is_capitalized
     Living weapon      living weapon            True
        Jump-start         jump-start            True
Commander ninjutsu commander ninjutsu            True
Legendary landwalk legendary landwalk            True
 Nonbasic landwalk  nonbasic landwalk            True
         Megamorph          megamorph            True
             Haunt              haunt            True
          Forecast           forecast            True
             Graft              graft            True
           Fortify            fortify            True
            Frenzy             frenzy            True
        Gravestorm         gravestorm            True
          Hideaway           hideaway            True
          Level Up           level up            True
            Infect             infect            True

✅ No duplicate mechanics found (case-insensitive)


## Search Pattern Testing

Test mechanic detection against sample Magic oracle texts using lowercase string matching.

In [8]:
# Test how we would search for these mechanics in oracle text
print("🔍 SEARCH PATTERN TESTING")

# Sample oracle texts to test against
sample_oracle_texts = [
    "Flying, deathtouch",
    "Whenever you proliferate, draw a card.",
    "Landfall — Whenever a land enters the battlefield under your control, create a token.",
    "Cycling {2}",
    "Suspend 3—{1}{U}",
    "Ninjutsu {U}",
    "This creature has flying and first strike.",
    "Investigate. (Create a Clue token.)",
    "Battalion — Whenever this creature attacks with two or more other creatures, gain 3 life."
]

# Test simple lowercase matching
print("\n📝 Testing simple lowercase matching:")
for text in sample_oracle_texts:
    text_lower = text.lower()
    found_mechanics = []
    
    for mechanic in df['mechanic']:
        if mechanic.lower() in text_lower:
            found_mechanics.append(mechanic)
    
    print(f"\n  Text: {text[:60]}...")
    if found_mechanics:
        print(f"  Found: {', '.join(found_mechanics)}")
    else:
        print(f"  Found: (none)")

🔍 SEARCH PATTERN TESTING

📝 Testing simple lowercase matching:

  Text: Flying, deathtouch...
  Found: Deathtouch, Flying

  Text: Whenever you proliferate, draw a card....
  Found: Proliferate

  Text: Landfall — Whenever a land enters the battlefield under your...
  Found: Create, Landfall

  Text: Cycling {2}...
  Found: Cycling

  Text: Suspend 3—{1}{U}...
  Found: Suspend

  Text: Ninjutsu {U}...
  Found: Ninjutsu

  Text: This creature has flying and first strike....
  Found: First strike, Flying

  Text: Investigate. (Create a Clue token.)...
  Found: Create, Investigate

  Text: Battalion — Whenever this creature attacks with two or more ...
  Found: Battalion


## Edge Cases & Special Patterns

Identify and categorize edge cases: multi-word mechanics, very short mechanics, mechanics with numbers, and special characters.

In [9]:
# Identify edge cases
print("⚠️ EDGE CASES & SPECIAL PATTERNS")

# Multi-word mechanics
print(f"\n1. Multi-word mechanics (potential matching issues):")
multi_word_mechs = df[df['word_count'] > 1]['mechanic'].tolist()
print(f"   Count: {len(multi_word_mechs)}")
print(f"   Examples: {multi_word_mechs[:10]}")

# Very short mechanics (2-3 chars - might cause false positives)
short_mechs = df[df['length'] <= 3]['mechanic'].tolist()
if short_mechs:
    print(f"\n2. Very short mechanics (false positive risk):")
    print(f"   Count: {len(short_mechs)}")
    print(f"   All: {short_mechs}")
else:
    print(f"\n2. No very short mechanics found ✅")

# Mechanics with numbers
mechs_with_numbers = df[df['mechanic'].str.contains(r'\d', regex=True, na=False)]['mechanic'].tolist()
if mechs_with_numbers:
    print(f"\n3. Mechanics with numbers:")
    print(f"   Count: {len(mechs_with_numbers)}")
    print(f"   All: {mechs_with_numbers}")
else:
    print(f"\n3. No mechanics with numbers found ✅")

# Mechanics with special characters (beyond hyphen/space)
special_char_pattern = r'[^a-zA-Z\s\-]'
special_char_mechs = df[df['mechanic'].str.contains(special_char_pattern, regex=True, na=False)]['mechanic'].tolist()
if special_char_mechs:
    print(f"\n4. Mechanics with special characters:")
    print(f"   Count: {len(special_char_mechs)}")
    print(f"   All: {special_char_mechs}")
else:
    print(f"\n4. No mechanics with special characters found ✅")

⚠️ EDGE CASES & SPECIAL PATTERNS

1. Multi-word mechanics (potential matching issues):
   Count: 47
   Examples: ['Living weapon', 'Commander ninjutsu', 'Legendary landwalk', 'Nonbasic landwalk', 'Level Up', 'Double strike', 'Cumulative upkeep', 'First strike', 'Aura Swap', 'Battle Cry']

2. Very short mechanics (false positive risk):
   Count: 1
   All: ['Tap']

3. No mechanics with numbers found ✅

4. Mechanics with special characters:
   Count: 5
   All: ['For Mirrodin!', "Doctor's companion", "Council's dilemma", "Hero's Reward", 'Start your engines!']


## Proposed MongoDB Schema Preview

Display example normalized documents showing proposed MongoDB collection structure.

In [10]:
# Show what the normalized data structure would look like
print("💾 PROPOSED MONGODB SCHEMA PREVIEW")
print("\nIf we were to store this in MongoDB, the structure could be:")

normalized_example = []
for _, row in df.head(10).iterrows():
    normalized_example.append({
        'id': row['mechanic'].lower().replace(' ', '_').replace('-', '_'),
        'name': row['mechanic'],
        'category': row['category'],
        'search_pattern': row['mechanic'].lower(),
        'word_count': row['word_count'],
        'updated_at': '2026-01-22T00:00:00Z'
    })

print(json.dumps(normalized_example, indent=2))

print("\n📝 Collection name suggestion: 'mtg_mechanics'")
print("📝 Index suggestions:")
print("  - Unique index on 'id'")
print("  - Index on 'category' for filtering")
print("  - Text index on 'name' for search")

💾 PROPOSED MONGODB SCHEMA PREVIEW

If we were to store this in MongoDB, the structure could be:
[
  {
    "id": "living_weapon",
    "name": "Living weapon",
    "category": "keyword_ability",
    "search_pattern": "living weapon",
    "word_count": 2,
    "updated_at": "2026-01-22T00:00:00Z"
  },
  {
    "id": "jump_start",
    "name": "Jump-start",
    "category": "keyword_ability",
    "search_pattern": "jump-start",
    "word_count": 1,
    "updated_at": "2026-01-22T00:00:00Z"
  },
  {
    "id": "commander_ninjutsu",
    "name": "Commander ninjutsu",
    "category": "keyword_ability",
    "search_pattern": "commander ninjutsu",
    "word_count": 2,
    "updated_at": "2026-01-22T00:00:00Z"
  },
  {
    "id": "legendary_landwalk",
    "name": "Legendary landwalk",
    "category": "keyword_ability",
    "search_pattern": "legendary landwalk",
    "word_count": 2,
    "updated_at": "2026-01-22T00:00:00Z"
  },
  {
    "id": "nonbasic_landwalk",
    "name": "Nonbasic landwalk",
    "cate

## Summary & Recommendations

Generate comprehensive summary with API validation results, key findings, potential issues, and next steps.

In [11]:
print("=" * 80)
print("📊 SUMMARY & RECOMMENDATIONS")
print("=" * 80)

print(f"\n✅ API VALIDATION:")
print(f"  - Keyword Abilities: {len(abilities_result['data']) if abilities_result['success'] else 'FAILED'}")
print(f"  - Keyword Actions: {len(actions_result['data']) if actions_result['success'] else 'FAILED'}")
print(f"  - Ability Words: {len(ability_words_result['data']) if ability_words_result['success'] else 'FAILED'}")
print(f"  - Total Mechanics: {len(df)}")

print(f"\n💡 KEY FINDINGS:")
print(f"  1. All catalogs use consistent format (list of strings)")
print(f"  2. Casing: {'Capitalized' if df.iloc[0]['mechanic'][0].isupper() else 'Lowercase'}")
print(f"  3. Multi-word mechanics: {len(df[df['word_count'] > 1])} found")
print(f"  4. Longest mechanic: {df['length'].max()} characters")
print(f"  5. Shortest mechanic: {df['length'].min()} characters")

print(f"\n🚨 POTENTIAL ISSUES:")
issues = []

if len(df[df['word_count'] > 1]) > 10:
    issues.append(f"  - {len(df[df['word_count'] > 1])} multi-word mechanics require careful matching")

if duplicates:
    issues.append(f"  - Found {len(duplicates)} case-sensitive duplicates")

if len(df[df['length'] <= 3]) > 0:
    issues.append(f"  - {len(df[df['length'] <= 3])} very short mechanics may cause false positives")

if issues:
    for issue in issues:
        print(issue)
else:
    print("  ✅ No major issues detected")

print(f"\n📋 NEXT STEPS:")
print(f"  1. ⏸️  STOP HERE - Review findings with user")
print(f"  2. 🗄️  Design MongoDB schema based on validated structure")
print(f"  3. 🔄 Plan integration with card scoring algorithm")
print(f"  4. 🧪 Test mechanic detection on real card oracle text")

print("\n✅ Exploration complete - awaiting user review")

📊 SUMMARY & RECOMMENDATIONS

✅ API VALIDATION:
  - Keyword Abilities: 216
  - Keyword Actions: 70
  - Ability Words: 66
  - Total Mechanics: 352

💡 KEY FINDINGS:
  1. All catalogs use consistent format (list of strings)
  2. Casing: Capitalized
  3. Multi-word mechanics: 47 found
  4. Longest mechanic: 30 characters
  5. Shortest mechanic: 3 characters

🚨 POTENTIAL ISSUES:
  - 47 multi-word mechanics require careful matching
  - 1 very short mechanics may cause false positives

📋 NEXT STEPS:
  1. ⏸️  STOP HERE - Review findings with user
  2. 🗄️  Design MongoDB schema based on validated structure
  3. 🔄 Plan integration with card scoring algorithm
  4. 🧪 Test mechanic detection on real card oracle text

✅ Exploration complete - awaiting user review


## Export Sample Data

Export analysis results as CSV and complete API responses as JSON files to the notebooks folder.

In [12]:
# Export sample data for reference
df.to_csv('/workspaces/mtgecorec/notebooks/scryfall_mechanics_sample.csv', index=False)

# Export full lists as JSON
export_data = {
    'keyword_abilities': abilities_result['data'] if abilities_result['success'] else [],
    'keyword_actions': actions_result['data'] if actions_result['success'] else [],
    'ability_words': ability_words_result['data'] if ability_words_result['success'] else [],
    'fetched_at': '2026-01-22T00:00:00Z',
    'total_count': len(df)
}

with open('/workspaces/mtgecorec/notebooks/scryfall_mechanics_full.json', 'w') as f:
    json.dump(export_data, f, indent=2)

print("✅ Sample data exported:")
print("  - scryfall_mechanics_sample.csv")
print("  - scryfall_mechanics_full.json")

✅ Sample data exported:
  - scryfall_mechanics_sample.csv
  - scryfall_mechanics_full.json
